<a href="https://colab.research.google.com/github/laloflogar/RB2026/blob/main/BEA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Importaciones

In [ ]:
#bloque de código 1

import numpy as np
import pandas as pd
import requests
from datetime import datetime
import plotly.graph_objects as go
from google.colab import data_table
data_table.enable_dataframe_formatter()

URL común para todas las consultas y userID personal (puedes obtener tu propio user ID registrandote en la página del BEA)

In [ ]:
#bloque de código 2

URL = 'https://apps.bea.gov/api/data/'
USER_ID = 'A7B059F6-0E57-4DC7-B774-8F19787455D9'

PARAMS = {
        'UserID': USER_ID,
        'Method': 'GetData',
        'ResultFormat': 'JSON'
    }

Este es el código que hace la conexión con la página del BEA. No es necesario hacerle mucho caso y está prohibid moverle a menos de que seas muy osado.

In [ ]:
#bloque de código 3
def get_data_from_bea(params):
    try:
        response = requests.get(URL, params=params)
        if response.status_code != 200:
            print(f"Error de conexión: {response.status_code}")
            return None

        raw_data = response.json()

        if 'BEAAPI' in raw_data and 'Results' in raw_data['BEAAPI']:
            results = raw_data['BEAAPI']['Results']

            # Ajuste para GDPbyIndustry: A veces Results es una lista
            if isinstance(results, list):
                results = results[0]

            if 'Error' in results:
                print(f"Error BEA: {results['Error']['APIErrorDescription']}")
                return None

            if 'Data' in results:
                df = pd.DataFrame(results['Data'])

                # Estandarizar nombres de columnas a Minúsculas para evitar errores
                df.columns = [c.lower() for c in df.columns]

                # Limpieza de DataValue
                if 'datavalue' in df.columns:
                    df['datavalue'] = df['datavalue'].str.replace(',', '').astype(float)

                return df
            else:
                print("No se encontraron nodos de datos.")
                return None
    except Exception as e:
        print(f"Ocurrió un error: {e}")
        return None

Aquí viene la parte chistosa. Como cada dataset requiere diferentes parámetros, aquí hay una lista de diferentes funciones para cada uno. Solo es necesario que ejecutes los bloques de código que necesitas para cada función y para cada ejecución (después de los bloques de código #1 #2 Y #2)

Hay más información sobre todos los datasets en este manualote [https://apps.bea.gov/api/_pdf/bea_web_service_api_user_guide.pdf](https://apps.bea.gov/api/_pdf/bea_web_service_api_user_guide.pdf)

---

Primero debemos de conocer cuales datasets podemos usar

In [ ]:
def obtener_lista_datasets():
    # Parámetros obligatorios para este método específico
    params = {
        'UserID': userID,
        'Method': 'GETDATASETLIST',
        'ResultFormat': 'JSON'
    }

    try:
        response = requests.get(url, params=params)
        raw_data = response.json()

        if 'BEAAPI' in raw_data and 'Results' in raw_data['BEAAPI']:
            datasets = raw_data['BEAAPI']['Results']['Dataset']
            return pd.DataFrame(datasets)
        else:
            print("No se pudo recuperar la lista.")
            return None

    except Exception as e:
        print(f"Error: {e}")
        return None

df_datasets = obtener_lista_datasets()

if df_datasets is not None:
    print("Datasets disponibles en BEA :")
    print(df_datasets[['DatasetName', 'DatasetDescription']])

Datasets disponibles en BEA :
                DatasetName                                 DatasetDescription
0                      NIPA                               Standard NIPA tables
1        NIUnderlyingDetail               Standard NI underlying detail tables
2                       MNE                          Multinational Enterprises
3               FixedAssets                       Standard Fixed Assets tables
4                       ITA                International Transactions Accounts
5                       IIP                  International Investment Position
6               InputOutput                                  Input-Output Data
7             IntlServTrade                       International Services Trade
8               IntlServSTA  International Services Supplied Through Affili...
9             GDPbyIndustry                                    GDP by Industry
10                 Regional                                 Regional data sets
11  UnderlyingGDPbyInd

---

**Consulta de Dataset: NIPA**

Parámetros obligatorios:

*  tableName: (ej. 'T10101' para el PIB).
*  frequency: 'A' (Anual), 'Q' (Trimestral), 'M' (Mensual).
*  year: 'ALL' o una lista de años (ej. '2020,2021,2022').


In [ ]:
def descarga_nipa(table_name, frequency='A', year='ALL'):

    p = PARAMS.copy()
    p.update({
        'DataSetName': 'NIPA',
        'TableName': table_name,
        'Frequency': frequency,
        'Year': year
    })

    return get_data_from_bea(p)

Código para saber qué hace cada Line Number

In [ ]:
df_nipa = descarga_nipa('T10101', 'Q', 'ALL')

if df_nipa is not None:
  print(df_nipa[['linenumber', 'linedescription']].drop_duplicates())
  #LOS NOMBRES ESTÁN EN MINÚSCULAS PQ ASÍ LOS CONVERTIMOS EN LA FUNCION ANTERIOR PQ LOS QUE HICIERON LAS TABLAS SON TONTOS Y A VECES PONEN LETRAS EN MAYUSCULAS DONDE NO DEBERÍAN DE IR

     linenumber                                    linedescription
0             1                             Gross domestic product
314           2                  Personal consumption expenditures
628           3                                              Goods
942           4                                      Durable goods
1256          5                                   Nondurable goods
1570          6                                           Services
1884          7                  Gross private domestic investment
2198          8                                   Fixed investment
2512          9                                     Nonresidential
2826         10                                         Structures
3140         11                                          Equipment
3454         12                     Intellectual property products
3768         13                                        Residential
4082         16                                            Exp

Código de consulta. MODIFICABLE

In [ ]:
# --- EJEMPLO DE USO ---
#Descargamos la tabla T10101 (GDP por componentes) en frecuencia Trimestral

df_nipa = descarga_nipa('T10101', 'Q', 'ALL')

if df_nipa is not None:
    # Como la tabla trae muchos indicadores, filtramos por el que queremos:
    # LineNumber '1' es "Gross Domestic Product"
    pib_series = df_nipa[df_nipa['linenumber'] == '1'].copy()

    # Ahora que es una serie única, podemos poner el índice de tiempo
    pib_series.set_index('timeperiod', inplace=True)

    print("Datos del PIB (Dataset NIPA) descargados:")
    display(pib_series[['datavalue']])

    #para ver los últimos resultados, o sea los más recientes, puedes comentar la linea anterior y descomentar la siguiente:
    #print(pib_series[['DataValue']].tail(20))
    #el argumento de tail es la cantidad de registros a mostrar

Datos del PIB (Dataset NIPA) descargados:


,datavalue
timeperiod,
1947Q2,-1.0
1947Q3,-0.8
1947Q4,6.4
1948Q1,6.2
1948Q2,6.8
...,...
2024Q3,3.3
2024Q4,1.9
2025Q1,-0.6


---

**Consulta de Dataset: NIUnderlyingDetail**

Parámetros obligatorios:

*   tableName
*   frequency: 'A' (Anual), 'Q' (Trimestral), 'M' (Mensual).
*   year: 'ALL' o una lista de años (ej. '2020,2021,2022').



In [1]:
def descarga_ni_underlying(table_name, frequency='A', year='ALL'):
    p = PARAMS.copy()

    p.update({
        'DataSetName': 'NIUnderlyingDetail',
        'TableName': table_name,
        'Frequency': frequency,
        'Year': year
    })

    return get_data_from_bea(p)


Código para saber qué es cada Line Number

In [ ]:
df_underlying = descarga_nipa('T10101', 'Q', 'ALL')

if df_underlying is not None:
  print(df_underlying[['linenumber', 'linedescription']].drop_duplicates())

     linenumber                                    linedescription
0             1                             Gross domestic product
314           2                  Personal consumption expenditures
628           3                                              Goods
942           4                                      Durable goods
1256          5                                   Nondurable goods
1570          6                                           Services
1884          7                  Gross private domestic investment
2198          8                                   Fixed investment
2512          9                                     Nonresidential
2826         10                                         Structures
3140         11                                          Equipment
3454         12                     Intellectual property products
3768         13                                        Residential
4082         16                                            Exp

Código de consulta. MODIFICABLE

In [ ]:
# --- EJEMPLO DE USO ---
# Ejemplo: Gasto en consumo personal (U20305), mensual (M), para 2023 y 2024 [cite: 729]
df_underlying = descarga_ni_underlying('U20305', 'M', '2023,2024')

if df_underlying is not None:
    # Filtramos por una línea específica antes de poner el índice
    # La línea '1' es el total del indicador de la tabla
    detalles_series = df_underlying[df_underlying['linenumber'] == '1'].copy()

    # Establecemos el índice de tiempo (formato YYYYMX para mensuales)
    detalles_series.set_index('timeperiod', inplace=True)

    print("Datos de NIUnderlyingDetail descargados:")
    display(detalles_series[['datavalue']])

    #para ver los últimos resultados, o sea los más recientes, puedes comentar la linea anterior y descomentar la siguiente:
    #print(detalles_series[['DataValue']].tail(20))
    #el argumento de tail es la cantidad de registros a mostrar

Datos de NIUnderlyingDetail descargados:


,datavalue
timeperiod,
2023M01,18449085.0
2023M02,18491570.0
2023M03,18520420.0
2023M04,18641787.0
2023M05,18664600.0
2023M06,18772996.0
2023M07,18884215.0
2023M08,18953262.0
2023M09,19048067.0


---

**Consulta de Dataset: Regional**

Parámetros obligatorios:
* TableName: Published table name (page 58)
* LineCode: Es el identificador numérico del dato estadístico (similar al LineNumber de NIPA). Por ejemplo, en la tabla SAINC1, el LineCode '10' es el ingreso personal per cápita.
* GeoFips: The state, county or MSA code
* year: 'ALL' o una lista de años (ej. '2020,2021,2022').

In [ ]:
def descarga_regional(table_name, line_code, geo_fips='00000', year='ALL'):
    """
    Descarga datos del dataset Regional del BEA.

    Argumentos:
        table_name: Nombre de la tabla (ej. 'SAINC1' para ingresos personales estatales).
        line_code: Código de la línea estadística (ej. '10').
        geo_fips: Código de ubicación ('00000' = US, 'STATE' = todos los estados).
        year: 'ALL' o años específicos.
    """
    p = PARAMS.copy()
    p.update({
        'DataSetName': 'Regional',
        'TableName': table_name,
        'LineCode': line_code,
        'GeoFips': geo_fips,
        'Year': year
    })

    return get_data_from_bea(p)

*ESTE CÓDIGO DE ABAJO NO CUAJÓ PERO YA TENGO MUCHO SUIEÑO ADIÓS*

In [ ]:
# --- EJEMPLO DE USO ---

# Tabla SAINC1: Personal Income Summary
# LineCode 10: Per capita personal income (Ingreso per cápita)
# GeoFips STATE: Trae todos los estados para comparar
df_regional = descarga_regional(table_name='SAINC1', line_code='10', geo_fips='STATE', year='2022,2023')

if df_regional is not None:
    # 1. Normalizar nombres (nuestra función get_data_from_bea ya los pone en minúscula)
    # Las columnas clave en Regional suelen ser: 'geofips', 'geoname', 'datavalue', 'timeperiod'

    print("Primeros registros de datos regionales:")
    display(df_regional[['timeperiod', 'geofips', 'geoname', 'datavalue']].head(10))

    # 2. Ejemplo: Filtrar solo por el estado de California (GeoFips '06000')
    ca_fips = '06000'
    if ca_fips in df_regional['geofips'].values:
        df_ca = df_regional[df_regional['geofips'] == ca_fips].copy()
        df_ca.set_index('timeperiod', inplace=True)
        print(f"\nIngreso per cápita en {df_ca['geoname'].iloc[0]}:")
        display(df_ca[['datavalue']])

---

**Consulta de Dataset: GDPbyIndustry**
Parámetros obligatorios:

* tableId: The unique GDP by Industry table identifier. ALL for all.
* Industry:  List of industries to retrieve.
* frequency: 'A' (Anual), 'Q' (Trimestral).
* year: 'ALL' o una lista de años (ej. '2020,2021,2022').

In [ ]:
def descarga_gdp_industry(table_id, industry='ALL', frequency='A', year='ALL'):
    # Copiamos la base global PARAMS que ya tiene UserID, Method y ResultFormat
    p = PARAMS.copy()

    # Añadimos los parámetros específicos para este dataset
    p.update({
        'DataSetName': 'GDPbyIndustry',
        'TableID': table_id,
        'Industry': industry,   # 'ALL' para todas las industrias
        'Frequency': frequency,  # 'A' (Anual) o 'Q' (Trimestral)
        'Year': year            # 'ALL' o años específicos
    })

    # Enviamos a la función centralizada que ya creaste
    return get_data_from_bea(p)

Catálogo de industrias

In [ ]:
df_industrias = descarga_gdp_industry(table_id='1', industry='ALL', year='ALL')

if df_industrias is not None:
    # 1. Explorar catálogo (usando minúsculas)
    # Nota: la respuesta del API decía 'industrydescription', aquí ya está normalizado
    catalogo = df_industrias[['industry', 'industrydescription']].drop_duplicates()
    print("Catálogo de industrias encontradas:")
    display(catalogo)

Catálogo de industrias encontradas:


,industry,industrydescription
0,11,"Agriculture, forestry, fishing, and hunting"
1,111CA,Farms
2,113FF,"Forestry, fishing, and related activities"
3,21,Mining
4,211,Oil and gas extraction
...,...,...
95,ORE,Other real estate
96,PGOOD,Private goods-producing industries<sup>1</sup>
97,PROF,Professional and business services
98,PSERV,Private services-producing industries<sup>2</sup>


Código de consulta. MODIFICABLE.

In [ ]:
# --- EJEMPLO DE USO ---
df_industrias = descarga_gdp_industry(table_id='1', industry='ALL', year='ALL')

if df_industrias is not None:

    # 2. Filtrar por Manufactura (Código '31G' según tu log de depuración)
    # IMPORTANTE: En tu log, el código de "Manufacturing" es '31G', no '6'
    codigo_buscado = '31G'

    if codigo_buscado in df_industrias['industry'].values:
        df_sector = df_industrias[df_industrias['industry'] == codigo_buscado].copy()

        # El tiempo en este dataset se llama 'year' (o 'quarter' si es Q)
        df_sector.set_index('year', inplace=True)

        print(f"\nSerie de tiempo para: {df_sector['industrydescription'].iloc[0]}")
        display(df_sector[['datavalue']])
    else:
        print(f"\nEl código '{codigo_buscado}' no existe en esta tabla.")


Serie de tiempo para: Manufacturing


,datavalue
year,
1997,1382.9
1998,1430.6
1999,1489.6
2000,1549.8
2001,1473.5
2002,1468.3
2003,1524.0
2004,1607.8
2005,1692.5
